In [1]:
!python3 --version

Python 3.7.12


### Cloning the GitHub repository

In [ ]:
!git clone https://github.com/hugojair/mocca-at-icmi21.git

Cloning into 'mocca-at-icmi21'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 31 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), done.


### Unzipping Dataset

In [ ]:
import os
import sys

In [ ]:
PROJECT_DIR = r"/content/drive/MyDrive/Mini Project II/Dataset/"
TRAINING_DIR = os.path.join(PROJECT_DIR,'Training')
TEST_DIR = os.path.join(PROJECT_DIR,"Test")

import shutil

if os.path.exists(PROJECT_DIR):
    shutil.rmtree(PROJECT_DIR)

for dir in [PROJECT_DIR,TRAINING_DIR,TEST_DIR]:
    os.makedirs(dir)
    print('New Folder Created at {0}'.format(dir))

In [ ]:
!unzip mocca-at-icmi21/mocca-training.zip -d "/content/drive/MyDrive/Mini Project II/Dataset/Training/"
!unzip mocca-at-icmi21/test.zip -d "/content/drive/MyDrive/Mini Project II/Dataset/Test/"

### Downloading Video Dataset

In [3]:
!pip install -U pytube
!pip install -U get-video-properties

     |████████████████████████████████| 56 kB 2.3 MB/s 
     |████████████████████████████████| 45.1 MB 1.3 MB/s 


In [4]:
import os
import pytube
import sys
import pytube
import pandas as pd
import urllib.request
from videoprops import get_video_properties

In [5]:
PROJECT_DIR = r"/content/drive/MyDrive/Mini_Project_II/dataset"
TRAINING_DIR = os.path.join(PROJECT_DIR,'Training')
TEST_DIR = os.path.join(PROJECT_DIR,"Test")
DATASET_DIR = os.path.join(TRAINING_DIR,'videos')

In [6]:
print(DATASET_DIR)

/content/drive/MyDrive/Mini_Project_II/dataset/Training/videos


In [7]:
def downloadYoutube(filename,url):
    try:
        from pathlib import Path
        if not os.path.exists(Path(DATASET_DIR,'.'.join([filename,'mp4']))):
            yt = pytube.YouTube(url)
            video = yt.streams.get_highest_resolution()
            out_file = video.download(f'{DATASET_DIR}/')
            ext = out_file.split('.')[-1]
            yt_id_filename = '.'.join([filename,ext])
            os.rename(out_file,f'{DATASET_DIR}/'+yt_id_filename)
            print(yt_id_filename + " download successful")
        else:
            print('File already found.Skipping {0}.mp4'.format(filename))
    except Exception as e:
        print('Error downloading {0} : {1}'.format(filename,e))

In [8]:
def downloadIMDB(filename,url):
    from pathlib import Path
    if not os.path.exists(Path(DATASET_DIR,'.'.join([filename,'mp4']))):
        videoplayerlink=url
        html=""
        with urllib.request.urlopen(videoplayerlink) as response:
            html = response.read()
        html=str(html)
        script = html.split('"video/mp4')
        videoLink=script[1].split('},{')[0].split('"')[4][0:-2]

        urllib.request.urlretrieve(videoLink,os.path.join(DATASET_DIR,f'{filename}.mp4'))
    else:
        print('File Already Found.Skipping {0}.mp4'.format(filename))

In [9]:
if not os.path.exists(DATASET_DIR):
    os.mkdir(DATASET_DIR)
final_videos=[]

csvFile=pd.read_csv('/content/drive/MyDrive/Mini_Project_II/dataset/Dataset_ComicMischief_Training_Scene_Multiclass_Annotations.csv')  

def compute_frame_rate_from_string_fraction(fraction):
    num,den = fraction.split( '/' )
    result= float(num)/float(den)
    return result

In [10]:
import time

#### Main Script - Downloading Videos from both Youtube and IMDB
totalSuccessfullAttempts=0
missingVideos=[]
for i,URL in enumerate(csvFile["Video URL"]):
    time.sleep(0.25)
    if(i==0 or (csvFile["Video URL"][i-1] not in URL)):
        try:
            if not os.path.exists(DATASET_DIR+f'{csvFile["Video ID"][i]}.mp4'):
                if "youtube" in URL:
                    downloadYoutube(csvFile["Video ID"][i],URL)
            
                if "imdb" in URL:
                    downloadIMDB(csvFile["Video ID"][i],URL)
            else:
                print('File already found. Skipping {0}.mp4'.format(csvFile["Video ID"][i]))
                
            props = get_video_properties(os.path.join(DATASET_DIR,f"{csvFile['Video ID'][i]}.mp4"))
            print(f"{i}. Video ID:" +"\t"+csvFile["Video ID"][i] +"\t URL: \t"+ URL+"\t"+"Codec:"+"\t"+  props['codec_name']+"\t"+ "Resolution:"+"\t"+ str(props['width']),"x",str(props['height']) +"\t"+ "Avg Frame rate:" +"\t"+ str(compute_frame_rate_from_string_fraction(props['avg_frame_rate'])))
            totalSuccessfullAttempts+=1
        except Exception as e:
            print(f"{i}. Video ID: " +"\t"+csvFile["Video ID"][i] +"\t URL: \t"+ URL+"\t"+ "Unable to download video")
            print('Exception : ',e)
            missingVideos.append((csvFile['Video ID'][i],URL))
print("Total Successfull Attempts = ", totalSuccessfullAttempts)
print("Missing Videos = ", missingVideos)

File already found.Skipping v6IyejLa35s.mp4
0. Video ID:	v6IyejLa35s	 URL: 	https://www.youtube.com/watch?v=v6IyejLa35s	Codec:	h264	Resolution:	1280 x 720	Avg Frame rate:	25.0
File already found.Skipping uThDGzLhOXg.mp4
3. Video ID:	uThDGzLhOXg	 URL: 	https://www.youtube.com/watch?v=uThDGzLhOXg	Codec:	h264	Resolution:	640 x 360	Avg Frame rate:	29.97002997002997
File already found.Skipping tt8366502.mp4
6. Video ID:	tt8366502	 URL: 	https://www.youtube.com/watch?v=VYxDD1KyRxY	Codec:	h264	Resolution:	1280 x 720	Avg Frame rate:	30.0
File already found.Skipping tt8097306.mp4
9. Video ID:	tt8097306	 URL: 	https://www.youtube.com/watch?v=30957QU9A6k	Codec:	h264	Resolution:	1280 x 720	Avg Frame rate:	23.976023976023978
File already found.Skipping tt7734218.mp4
11. Video ID:	tt7734218	 URL: 	https://www.youtube.com/watch?v=idLxrjDjHUU	Codec:	h264	Resolution:	1280 x 720	Avg Frame rate:	23.976023976023978
File already found.Skipping tt7319064.mp4
14. Video ID:	tt7319064	 URL: 	https://www.youtub

In [11]:
missingVideos

[('tt0493405', 'https://www.youtube.com/watch?v=XiDPT1yCJ8Y'),
 ('kCppUtS9vLk', 'https://www.youtube.com/watch?v=kCppUtS9vLk'),
 ('EbcfiIeH63M', 'https://www.youtube.com/watch?v=EbcfiIeH63M'),
 ('pxxPznV38Hk', 'https://www.youtube.com/watch?v=pxxPznV38Hk'),
 ('1yhNm_8q07g', 'https://www.youtube.com/watch?v=1yhNm_8q07g')]

In [12]:
len(missingVideos)

5

In [ ]:
for i,file in enumerate(os.listdir(DATASET_DIR)):
    print(f'{i}. {file}')

0. tt1232829.mp4
1. tt3859304.mp4
2. tt0287969.mp4
3. tt2005164.mp4
4. tt1205558.mp4
5. tt2334871.mp4
6. tt2718440.mp4
7. tt0368343.mp4
8. tt0995039.mp4
9. tt0810922.mp4
10. tt2278388.mp4
11. tt2179116.mp4
12. tt2101441.mp4
13. tt1758795.mp4
14. tt0398839.mp4
15. tt1637725.mp4
16. v6IyejLa35s.mp4
17. uThDGzLhOXg.mp4
18. tt8366502.mp4
19. tt8097306.mp4
20. tt7734218.mp4
21. tt7319064.mp4
22. tt7256866.mp4
23. tt6516314.mp4
24. tt6251024.mp4
25. tt5879216.mp4
26. tt5716848.mp4
27. tt5688932.mp4
28. tt5162658.mp4
29. tt4769836.mp4
30. tt4738776.mp4
31. tt4503510.mp4
32. tt4156972.mp4
33. tt3856992.mp4
34. tt2917484.mp4
35. tt1959563.mp4
36. tt1524930.mp4
37. tt1349482.mp4
38. tt1220617.mp4
39. tt0066344.mp4
40. tMAwr-m4f3A.mp4
41. sRaF4gk1yhY.mp4
42. rvgYBbc6vYk.mp4
43. rqtN279OoX8.mp4
44. Rq1QAEyVK2w.mp4
45. ro2x8gd2v-M.mp4
46. QwcjToDEsUY.mp4
47. qr7onP8c8q8.mp4
48. psEHAKzMIvs.mp4
49. p3Co2OLO_eg.mp4
50. M2F7L2WBuKs.mp4
51. laqIl3LniQE.mp4
52. kltbeJ-tJrY.mp4
53. IUbn5ss8j9c.mp4
54. I-

In [13]:
len(set(csvFile['Video ID'].values))

347

### Generate 1 min scenes

In [14]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import os
import subprocess

SCENES_DIR = os.path.join(TRAINING_DIR,'scenes')

def split_video(file):
    input_file= os.path.join(DATASET_DIR,file)
    output_dir=os.path.join(SCENES_DIR,file.split('.')[0])
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    result = subprocess.run(["ffmpeg", "-i", input_file,"-map","0", "-c", "copy", "-f", "segment", "-segment_time", "60","-reset_timestamps", "1", output_dir+"/"+file.split(".")[0]+".%02d."+file.split(".")[-1]],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    
for video in os.listdir(DATASET_DIR):
    print(video)
    split_video(video)

v6IyejLa35s.mp4
uThDGzLhOXg.mp4
tt8366502.mp4
tt8097306.mp4
tt7734218.mp4
tt7319064.mp4
tt7256866.mp4
tt6516314.mp4
tt6251024.mp4
tt5879216.mp4
tt5716848.mp4
tt5688932.mp4
tt5162658.mp4
tt4769836.mp4
tt4738776.mp4
tt4503510.mp4
tt4156972.mp4
tt3856992.mp4
tt2917484.mp4
tt1959563.mp4
tt1524930.mp4
tt1349482.mp4
tt1220617.mp4
tt0066344.mp4
tMAwr-m4f3A.mp4
sRaF4gk1yhY.mp4
rvgYBbc6vYk.mp4
rqtN279OoX8.mp4
Rq1QAEyVK2w.mp4
ro2x8gd2v-M.mp4
QwcjToDEsUY.mp4
qr7onP8c8q8.mp4
psEHAKzMIvs.mp4
p3Co2OLO_eg.mp4
M2F7L2WBuKs.mp4
laqIl3LniQE.mp4
kltbeJ-tJrY.mp4
IUbn5ss8j9c.mp4
I-SFA5Xs0s8.mp4
IKQJZ93DhV8.mp4
hMKDu1LESQA.mp4
h3Fxl9GMjZI.mp4
Goq-yLTafws.mp4
gGTRdpW8oZA.mp4
FLOYmmFAsYI.mp4
FIGL4eaks20.mp4
era18XunpLc.mp4
EOAQuBtfNlc.mp4
ccIr0atau7M.mp4
BtMd6PbscwE.mp4
aBbzgUTWE6w.mp4
A0641hHG1IQ.mp4
49iNyQvMgy0.mp4
5ixoqCecD6Y.mp4
ZsWPgEpfafM.mp4
ZqEsUqI4YD8.mp4
zNjO6arjuAc.mp4
Z9QTS4doveY.mp4
z0hrMg1j_d4.mp4
YT127qw8eQQ.mp4
yqulWPljawo.mp4
yoA8KNDyIwU.mp4
Ym1NLvmcl0k.mp4
YLuZjpxmsZQ.mp4
YlhxwPxssYQ.mp4
YHTm

In [15]:
len(os.listdir(DATASET_DIR))

342

### Prepare Training Dataset

In [17]:
import os
import shutil
import pandas as pd


scenesDirectory= SCENES_DIR
trainingDataDirectory= os.path.join(TRAINING_DIR,'TrainingDataset')

if not os.path.exists(trainingDataDirectory):
    os.makedirs(trainingDataDirectory)
    
# csvFile=pd.read_csv('Dataset_ComicMischief_Training_Scene_Multiclass_Annotations.csv')  
csvFile=pd.read_csv('/content/drive/MyDrive/Mini_Project_II/dataset/Dataset_ComicMischief_Training_Scene_Multiclass_Annotations.csv')  


totalSuccessfullAttempts=0
totalFailedAttempts=0
for i,videoID in enumerate(csvFile["Video ID"]):
    try:
        sceneID = str(csvFile["Scene_ID"][i])
        shutil.copyfile(os.path.join(scenesDirectory,videoID,f'{videoID}.0{sceneID}.mp4'),os.path.join(trainingDataDirectory,f'{videoID}.0{sceneID}.mp4'))
        
        # shutil.copyfile(scenesDirectory+videoID+"/"+videoID+".0"+sceneID+".mp4", trainingDataDirectory+videoID+".0"+sceneID+".mp4")
        print("Video ID:",videoID,"Scene ID:",sceneID, "Successfully extracted")
        totalSuccessfullAttempts+=1
    except Exception as e:
        print("Video ID:",videoID,"Scene ID:",sceneID, "Failed")
        print('Exception : ',e)
        totalFailedAttempts+=1
print("Total Successfull Scene Extraction Attempts = ", totalSuccessfullAttempts)
print("Total Failed Scene Extraction Attempts = ", totalFailedAttempts)

Video ID: v6IyejLa35s Scene ID: 0 Successfully extracted
Video ID: v6IyejLa35s Scene ID: 1 Successfully extracted
Video ID: v6IyejLa35s Scene ID: 2 Successfully extracted
Video ID: uThDGzLhOXg Scene ID: 0 Successfully extracted
Video ID: uThDGzLhOXg Scene ID: 1 Successfully extracted
Video ID: uThDGzLhOXg Scene ID: 2 Successfully extracted
Video ID: tt8366502 Scene ID: 0 Successfully extracted
Video ID: tt8366502 Scene ID: 1 Successfully extracted
Video ID: tt8366502 Scene ID: 2 Successfully extracted
Video ID: tt8097306 Scene ID: 0 Successfully extracted
Video ID: tt8097306 Scene ID: 1 Successfully extracted
Video ID: tt7734218 Scene ID: 0 Successfully extracted
Video ID: tt7734218 Scene ID: 1 Successfully extracted
Video ID: tt7734218 Scene ID: 2 Successfully extracted
Video ID: tt7319064 Scene ID: 0 Successfully extracted
Video ID: tt7319064 Scene ID: 1 Successfully extracted
Video ID: tt7256866 Scene ID: 0 Successfully extracted
Video ID: tt7256866 Scene ID: 1 Successfully extracte